In [1]:
import requests #request package to retrieve content from a website
from bs4 import BeautifulSoup
import json

Definizione di due funzioni, rispettivamente per estrarre le domande e le risposte.
Per arrivare a queste conclusioni è stata fatta un'analisi preliminare del codice html delle pagine web del Ministero della salute contenenti domande e risposte (http://www.salute.gov.it/portale/nuovocoronavirus/archivioFaqNuovoCoronavirus.jsp).

In [2]:
# funzione per l'estrazione delle domande
def extract_questions(soup):
    questions = list()
    for q in soup.find_all('strong'):
        # controllo che mi serve per prendere effettivamente
        # solo le domande
        if (q.parent.name == 'button'): 
            s = q.string
            questions.append(s.strip())
    return (questions)

In [3]:
# funzione per l'estrazione delle risposte
def extract_answers(soup):
    answers = list()
    for div in soup.find_all('div', class_='collapse-body'):
        # trovo tutti i link nel div
        markup = div.find_all('a')
        for a in markup:
            # se il div ha qualche link
            if (a):
                if (a.get('href')[0] == "/"):
                    base_url = 'http://www.salute.gov.it'
                    link = base_url + a.get('href')
                else:
                    link = a.get('href')
                # cambio il testo contenuto tra <a> e </a>
                # con il link stesso
                a.string = a.string + "(" + link + ")"
        # a questo punto estraggo il testo (che contiene i link)
        # strip = True mi serve per pulirlo
        ans = div.get_text(" ", strip=True)
        answers.append(ans)
    return(answers)

Questa funzione è necessaria per estrapolare le risposte di una pagina web che presenta un codice leggermente diverso dalle altre.

In [4]:
# altra funzione per l'estrazione delle risposte
def extract_answers2(soup):
    answers = list()
    for div in soup.find_all('div', class_='collapse-body'):
        # trovo tutti i link nel div
        markup = div.find_all('a')
        for a in markup:
            # se il div ha qualche link
            if (a):
                if (a.get('href')[0] == "/"):
                    base_url = 'http://www.salute.gov.it'
                    link = base_url + a.get('href')
                else:
                    link = a.get('href')
                # cambio il testo contenuto tra <a> e </a>
                # con il link stesso
                if(a.string != None):
                    a.string = a.string + "(" + link + ")"
        # a questo punto estraggo il testo (che contiene i link)
        # strip = True mi serve per pulirlo
        ans = div.get_text(" ", strip=True)
        answers.append(ans)
    return(answers)

In [5]:
# funzione per estrarre i link delle pagine di cui effettuare lo scraping 
# delle domande e risposte
def extract_link(soup):
    links = list()
    # url da integrare con quello presente nell'href
    base_url = 'http://www.salute.gov.it'
    # <ul> con class_='nav navgoco' è una delle liste che contiene gli url che voglio
    # di queste <ul> ce ne sono due e io sono interessata alla seconda, quindi agisco
    # alla seconda iterazione (quando i = 1)
    i = 0
    for ul in soup.find_all('ul', class_ = 'nav navgoco'):
        if (i == 1):
            markup = ul.find_all('a')
            for a in markup:
                l = base_url+a.get('href')
                links.append(l)
        i += 1
    return links

In [6]:
# fornisco un unico url di partenza
url = 'http://www.salute.gov.it/portale/nuovocoronavirus/dettaglioFaqNuovoCoronavirus.jsp?lingua=italiano&id=231'
soup = BeautifulSoup(requests.get(url).text)
# estraggo i link delle pagine
links = extract_link(soup)

q_a = dict()
for l in links:
    url = l
    # se l'url in questione è quello indicato,
    # applico la seconda funzione per l'estrazione delle risposte
    if url == 'http://www.salute.gov.it/portale/nuovocoronavirus/dettaglioFaqNuovoCoronavirus.jsp?lingua=italiano&id=237':
        soup = BeautifulSoup(requests.get(url).text)
        questions = extract_questions(soup)
        answers = extract_answers2(soup)
    else:
        soup = BeautifulSoup(requests.get(url).text)
        questions = extract_questions(soup)
        answers = extract_answers(soup)
    # metto tutto dentro una struttura dati (dizionario)
    for q, a in zip (questions, answers):
        q_a[q] = (a, url)

q_a = dizionario che ha per chiave la domanda e come valore una tupla. Il primo elemento della tupla è la risposta alla domanda e il secondo elemento è il link dal quale le informazioni sono state estratte.

In [7]:
len(q_a) # numero di FAQ in nostro possesso

154

In [8]:
# salvataggio della struttura dati in un file esterno
with open('q&a.json', 'w') as json_file:
    json.dump(q_a, json_file)